Competition Link: https://www.kaggle.com/competitions/otto-recommender-system/overview
Using user data generated by users interacting on the website, we have to predict which item the user will click, put to cart and make an order.

Training Data is over 10GB which was very difficult to train on the cloud provided by Kaggle (after a few hours, the trainning got stuck), so I had to train and predict by chunk (fold). Model used was XGBoost classifier.

I also had to make Covisitation Matrix of all users to infer user behavior from using data from other users.

In [1]:
!pip install dask_cuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 1.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 822.1/822.1 kB 39.2 MB/s eta 0:00:00
  Attempting uninstall: dask
    Found existing installation: dask 2022.2.0
    Uninstalling dask-2022.2.0:
      Successfully uninstalled dask-2022.2.0
  Attempting uninstall: distributed
    Found existing installation: distributed 2022.2.0
    Uninstalling distributed-2022.2.0:
      Successfully uninstalled distributed-2022.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-cudf 21.10.1 requires cupy-cuda114, which is not installed.
dask-cudf 21.10.1 requires dask==2021.09.1, but you have dask 2022.1.0 which is incompatible.
dask-cudf 21.10.1 requires distributed==2021.09.1, but you have distributed 20

In [45]:
!pip install dask-ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.5/148.5 kB 2.1 MB/s eta 0:00:00a 0:00:01
^C
ERROR: Operation cancelled by user


In [2]:
cuda = 1

import numpy as np
import pandas as pd
import pickle, glob, gc
import xgboost as xgb
import dask.array as da
import dask.distributed
import dask.dataframe as dask1
from sklearn.model_selection import GroupKFold
#import dask_ml

import ctypes
import _pickle as pickle

if cuda == 1:
    import dask_cuda
    import cupy as cp
    from dask_cuda import LocalCUDACluster
    import dask_cudf
    import cudf
    
#Prediction
files = glob.glob('/kaggle/input/test-candidate-for-otto/cd_test_clicks*')
files2 = glob.glob('/kaggle/input/test-candidate-for-otto/cd_test_buys*')


'''
df_chunk_cudf = []
for f in files:
    df_cudf = dask_cudf.read_parquet(f)
    #df_cudf = cudf.read_parquet(f)
    df_chunk_cudf.append(df_cudf)
    cd_test_clicks = dask_cudf.concat(df_chunk_cudf)
'''
df_chunk_cudf = []
for f in files2:
    df_cudf = dask_cudf.read_parquet(f)
    #df_cudf = cudf.read_parquet(f)
    df_chunk_cudf.append(df_cudf)
    cd_test_buys = dask_cudf.concat(df_chunk_cudf)




In [47]:
%%time
dask__ = 0
cudf__ = 1

files = glob.glob('/kaggle/input/ottofeaturesengineering/cd_valid_clicks*')
#files = glob.glob('/kaggle/input/ottofeaturesengineering/cd_with_features_valid*')
df_chunk_cudf = []
df_chunk_dask = []
for f in files:
    if dask__ == 1:
        df_dask = dask1.read_parquet(f)
        df_chunk_dask.append(df_dask)
    if cudf__ == 1:
        df_cudf = dask_cudf.read_parquet(f)
        #df_cudf = cudf.read_parquet(f)
        df_chunk_cudf.append(df_cudf)

if cudf__ == 1:
    val_clicks_cd = dask_cudf.concat(df_chunk_cudf)
    #val_clicks_cd = cudf.concat(df_chunk_cudf)
    val_clicks_cd.repartition(partition_size='100MB')
    val_clicks_cd = val_clicks_cd.set_index(val_clicks_cd.index, sorted=True)
    val_clicks_cd = val_clicks_cd[['user', 'item', 'item_item_count',
    'item_user_count', 'item_mean_ratio', 'item_click_count', 'item_click2cart_ratio',
    'user_user_count', 'user_item_count', 'user_mean_ratio', 'user_click_count',
    'user_click2cart_ratio', 'interact_clicked', 'interact_carted', 'interact_ordered', 'clicks']]

if dask__ == 1:
    val_clicks_cd_dask = dask1.multi.concat(df_chunk_dask)
    val_clicks_cd_dask.repartition(partition_size='100MB')
    val_clicks_cd_dask = val_clicks_cd_dask.set_index(val_clicks_cd_dask.index, sorted=True)
    val_clicks_cd_dask = val_clicks_cd_dask[['user', 'item', 'clicks']]

KeyboardInterrupt: 

In [48]:
%%time
dask__ = 0
cudf__ = 1

files = glob.glob('/kaggle/input/ottofeaturesengineering/cd_valid_buys*')
#files = glob.glob('/kaggle/input/ottofeaturesengineering/cd_with_features_valid*')
df_chunk_cudf = []
df_chunk_dask = []
for f in files:
    if dask__ == 1:
        df_dask = dask1.read_parquet(f)
        df_chunk_dask.append(df_dask)
    if cudf__ == 1:
        df_cudf = dask_cudf.read_parquet(f)
        #df_cudf = cudf.read_parquet(f)
        df_chunk_cudf.append(df_cudf)

if cudf__ == 1:
    val_buys_cd = dask_cudf.concat(df_chunk_cudf)
    #val_buys_cd = cudf.concat(df_chunk_cudf)
    val_buys_cd.repartition(partition_size='100MB')
    val_buys_cd = val_buys_cd.set_index(val_buys_cd.index, sorted=True)
    val_buys_cd = val_buys_cd[['user', 'item', 'item_item_count',
    'item_user_count', 'item_mean_ratio', 'item_order_count', 'item_cart2order_ratio',
    'user_user_count', 'user_item_count', 'user_mean_ratio', 'user_order_count',
    'user_click2order_ratio', 'interact_clicked', 'interact_carted', 'interact_ordered',
    'orders']]

if dask__ == 1:
    val_buys_cd_dask = dask1.multi.concat(df_chunk_dask)
    val_buys_cd_dask.repartition(partition_size='100MB')
    val_buys_cd_dask = val_buys_cd_dask.set_index(val_buys_cd_dask.index, sorted=True)
    val_buys_cd_dask = val_buys_cd_dask[['user', 'item', 'carts', 'orders']]

KeyboardInterrupt: 

In [49]:
%%time
#Saving kfold group and index to files
kfold = {}
skf = GroupKFold(n_splits=5)

for fold,(train_idx, valid_idx) in enumerate(skf.split(val_clicks_cd_dask, val_clicks_cd_dask['clicks'], groups=val_clicks_cd_dask['user'])):
        kfold[f'train_idx{fold}'] = cp.array(train_idx)
        kfold[f'valid_idx{fold}'] = cp.array(valid_idx)
        
del val_clicks_cd_dask

NameError: name 'val_clicks_cd_dask' is not defined

In [50]:
%%time
#Saving kfold group and index to files
kfold = {}
skf = GroupKFold(n_splits=5)     
for fold,(train_idx, valid_idx) in enumerate(skf.split(val_buys_cd_dask, val_buys_cd_dask['carts'], groups=val_buys_cd_dask['user'])):
        kfold[f'train_idx{fold}'] = train_idx
        kfold[f'valid_idx{fold}'] = valid_idx

NameError: name 'val_buys_cd_dask' is not defined

In [51]:
%%time
#save file as pickle
import _pickle as pickle
with open("kfold_carts.pkl", "wb") as f:
    pickle.dump(kfold, f)

CPU times: user 231 µs, sys: 0 ns, total: 231 µs
Wall time: 243 µs


In [53]:
%%time

#Train using Dask DF
def trim_memory() -> int:
    libc = ctypes.CDLL("libc.so.6")
    return libc.malloc_trim(0)

def cudftodask(cudf_):
    return dask_cudf.from_cudf(cudf_, npartitions = 10)

def training(cd, features, type_, valid_ = True):

    client.run(gc.collect)
    gc.collect()
    
    if valid_ == True:
        for fold in range(4):
            fold += 1
            train_idx, valid_idx = client.scatter(kfold[f'train_idx{str(fold)}'], broadcast = True), client.scatter(kfold[f'valid_idx{str(fold)}'], broadcast = True)
            print(f'Progressing fold {str(fold)}')
                
            qid = client.gather([train_idx, valid_idx])
            print('Getting train and valid qid groupings...')
            train_qid = client.scatter(cd.loc[qid[0].get(), 'user'].compute(), broadcast = True)
            valid_qid = client.scatter(cd.loc[qid[1].get(), 'user'].compute(), broadcast = True)
            #qid = client.submit(f_, [train_qid, valid_qid])
            #train_qid = client.submit(cp.array, future_qid)

            print(f'Constructing X y train for fold {str(fold)}')
            X_train = client.scatter(cd.loc[qid[0].get(), features].compute(), broadcast = True)
            y_train = client.scatter(cd.loc[qid[0].get(), type_].compute(), broadcast = True)
            #training = client.submit(f_, [X_train, y_train])

            print(f'Constructing X y valid for fold {str(fold)}')
            X_valid = client.scatter(cd.loc[qid[1].get(), features].compute(), broadcast = True)
            y_valid = client.scatter(cd.loc[qid[1].get(), type_].compute(), broadcast = True)
            #valid = client.submit(f_, [X_valid, y_valid])
                
            gc.collect()
            client.run(gc.collect)

            print(f'Constructing DeskDeviceQuantileDMatrix for training for fold {str(fold)}')
            print('Gathering training dataframe...')
            train_results = client.gather([X_train, y_train, train_qid])
            print('Begin constructing...')
            dtrain = xgb.dask.DaskDeviceQuantileDMatrix(client, cudftodask(train_results[0]), cudftodask(train_results[1]), qid=cudftodask(train_results[2]))

            gc.collect()
            client.run(gc.collect)

            print(f'Constructing DeskDeviceQuantileDMatrix for valid fold {str(fold)}')
            print('Gathering valid dataframe...')
            valid_results = client.gather([X_valid, y_valid, valid_qid])
            print('Begin constructing...')
            dvalid = xgb.dask.DaskDeviceQuantileDMatrix(client, cudftodask(valid_results[0]), cudftodask(valid_results[1]), qid=cudftodask(valid_results[2]))

            print(f'Start training for fold {str(fold)}')
            gc.collect()
            client.run(gc.collect)
            xgb_parms = {'objective':'rank:pairwise', 'tree_method':'gpu_hist'}
            model = xgb.dask.train(client, xgb_parms, 
                dtrain=dtrain,
                evals=[(dtrain,'train'),(dvalid,'valid')],
                num_boost_round=1300,
                verbose_eval=100)
            booster = model['booster']
            history = model['history']
            booster.save_model(f'XGB_fold{fold}_{type_}.xgb')
            gc.collect()

    if valid_ == False:
        print('Getting train qid groupings...')
        train_qid = client.scatter(cd.loc[:, 'user'].compute(), broadcast = True)
            
        print(f'Constructing X y train for fold {str(fold)}')
        X_train = client.scatter(cd.loc[:, features].compute(), broadcast = True)
        y_train = client.scatter(cd.loc[:, type_].compute(), broadcast = True)
        #training = client.submit(f_, [X_train, y_train])

        gc.collect()
        client.run(gc.collect)

        print(f'Constructing DeskDeviceQuantileDMatrix for training')
        print('Gathering training dataframe...')
        train_results = client.gather([X_train, y_train, train_qid])
        print('Begin constructing...')
        dtrain = xgb.dask.DaskDeviceQuantileDMatrix(client, cudftodask(train_results[0]), cudftodask(train_results[1]), qid=cudftodask(train_results[2]))
            
        print(f'Start training...')
        gc.collect()
        client.run(gc.collect)
        xgb_parms = {'objective':'rank:pairwise', 'tree_method':'gpu_hist'}
        model = xgb.dask.train(client, xgb_parms, 
            dtrain=dtrain,
            evals=[(dtrain,'train')],
            num_boost_round=1300,
            verbose_eval=100)
        booster = model['booster']
        history = model['history']
        booster.save_model(f'XGB_{type_}.xgb')
        print('Training evaluation history:', history)
        gc.collect()

features = val_buys_cd.columns[2:-1]
with LocalCUDACluster(CUDA_VISIBLE_DEVICES = '0,1',
                      device_memory_limit=0.9, rmm_pool_size=0.9) as cluster:
    with dask.distributed.Client(cluster) as client:
        gc.collect()
        #client.run(trim_memory)
        client.run(cudf.set_allocator, "managed")
        client
        model = training(val_buys_cd, features, 'orders')

NameError: name 'val_buys_cd' is not defined

In [52]:
%%time
with open("/kaggle/input/kfold-for-otto/kfold_carts.pkl", "rb") as f:
    kfold = pickle.load(f)
    for fold in range(5):
        kfold[f'train_idx{str(fold)}'], kfold[f'valid_idx{str(fold)}'] = cp.asarray(kfold[f'train_idx{str(fold)}']), cp.asarray(kfold[f'valid_idx{str(fold)}'])

KeyboardInterrupt: 

In [6]:
#click
FEATURES_click = ['item_item_count',
    'item_user_count', 'item_mean_ratio', 'item_click_count', 'item_click2cart_ratio',
    'user_user_count', 'user_item_count', 'user_mean_ratio', 'user_click_count',
    'user_click2cart_ratio', 'interact_clicked', 'interact_carted', 'interact_ordered']
    
#cart
FEATURES_cart = ['item_item_count',
    'item_user_count', 'item_mean_ratio', 'item_cart_count', 'item_click2cart_ratio',
    'user_user_count', 'user_item_count', 'user_mean_ratio', 'user_cart_count',
    'user_click2cart_ratio', 'interact_clicked', 'interact_carted', 'interact_ordered']

#orders
FEATURES_orders = ['item_item_count',
    'item_user_count', 'item_mean_ratio', 'item_order_count', 'item_cart2order_ratio',
    'user_user_count', 'user_item_count', 'user_mean_ratio', 'user_order_count',
    'user_click2order_ratio', 'interact_clicked', 'interact_carted', 'interact_ordered']

def prediction(testdf, type_, fold_):
    preds = np.zeros(len(testdf))
    #qid = client.scatter(testdf.user, broadcast = True)
    for fold in range(fold_):
        print(f'loading {type_} model of fold {fold}')
        model = xgb.Booster()
        model.load_model(f'/kaggle/input/otto-covisitation-matrix/XGB_fold{fold}_{type_}.xgb')
        model.set_param({'predictor': 'gpu_predictor'})
        print(f'Creating DMatrix for fold {fold} of {type_}')
        dtest = xgb.dask.DaskDeviceQuantileDMatrix(client, testdf[FEATURES], qid = testdf.user)
        print('Predicting...')
        preds += (xgb.dask.predict(client, model, dtest)/fold_).compute()
        gc.collect()
        client.run(gc.collect)
    predictions = testdf[['user','item']].copy().compute()
    predictions['pred'] = preds
    return predictions

def processing(predictions):
    predictions = predictions.sort_values(['user','pred'], ascending=[True,False]).reset_index(drop=True)
    predictions['n'] = predictions.groupby('user').item.cumcount().astype('int8')
    predictions = predictions.loc[predictions.n<20]
    sub = predictions.groupby('user').item.apply(list)
    sub = sub.to_frame().reset_index()
    sub.item = sub.item.apply(lambda x: " ".join(map(str,x)))
    sub.columns = ['session_type','labels']
    sub.session_type = sub.session_type.astype('str')+ f'_{type_}'
    return sub

with LocalCUDACluster(CUDA_VISIBLE_DEVICES = '0,1',
                      device_memory_limit=0.9, rmm_pool_size=0.9) as cluster:
    with dask.distributed.Client(cluster) as client:
        client.run(cudf.set_allocator, "managed")
        FEATURES = FEATURES_carts
        clicks_sub = prediction(cd_test_clicks, 'clicks', 5)
        FEATURES = FEATURES_cart
        carts_sub = prediction(cd_test_buys, 'carts', 3)
        FEATURES = FEATURES_orders
        orders_sub = prediction(cd_test_buys, 'orders', 2)

distributed.preloading - INFO - Import preload module: dask_cuda.initialize
distributed.preloading - INFO - Import preload module: dask_cuda.initialize


loading orders model of fold 0
Creating DMatrix for fold 0 of orders
Predicting...
loading orders model of fold 1
Creating DMatrix for fold 1 of orders
Predicting...


In [7]:
def processing(predictions,type_):
    predictions = predictions.sort_values(['user','pred'], ascending=[True,False]).reset_index(drop=True)
    predictions['n'] = predictions.groupby('user').item.cumcount().astype('int8')
    predictions = predictions.loc[predictions.n<20]
    sub = predictions.groupby('user').item.apply(list)
    sub = sub.to_frame().reset_index()
    sub.item = sub.item.apply(lambda x: " ".join(map(str,x)))
    sub.columns = ['session_type','labels']
    sub.session_type = sub.session_type.astype('str')+ f'_{type_}'
    return sub

orders_sub = processing(orders_sub.to_pandas(), 'orders')
orders_sub.to_parquet('orders_sub_r.pqt')

In [9]:
clicks_sub = pd.read_parquet('/kaggle/input/kfold-for-otto/clicks_sub_r.pqt')

In [10]:
pred_df = pd.concat([clicks_sub, orders_sub, carts_sub])
pred_df.to_csv("submission.csv", index=False)
pred_df.head()

,session_type,labels
0,12899779_clicks,59625 3295 967363 225209 1790770 1138236 73169...
1,12899780_clicks,1551213 1344758 1142000 797055 108125 973453 7...
2,12899781_clicks,199008 194067 918667 528496 1495531 618888 172...
3,12899782_clicks,1494780 595994 740494 476063 413962 229748 654...
4,12899783_clicks,255297 300127 1729553 1754419 198385 1114789 1...


In [12]:
pred_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5015409 entries, 0 to 1671802
Data columns (total 2 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   session_type  object
 1   labels        object
dtypes: object(2)
memory usage: 114.8+ MB


Disregard the codes below this cell.
It was the code I wrote that did not work but kept for reference

In [ ]:
def f_(_):
    return _

def cudftodask(cudf_):
    return dask_cudf.from_cudf(cudf_, npartitions = 10)

features = val_clicks_cd.columns[2:-1]
with LocalCUDACluster(CUDA_VISIBLE_DEVICES = '0,1',
                      device_memory_limit=0.9, rmm_pool_size=0.9) as cluster:
    with dask.distributed.Client(cluster) as client:
        qid = client.scatter(val_clicks_cd.loc[kfold['valid_idx0'][0:100000].get(), 'user'].compute(), broadcast = True)
        t1 = client.scatter(val_clicks_cd.loc[kfold['valid_idx0'][0:100000].get(), features].compute(), broadcast = True)
        t2 = client.scatter(val_clicks_cd.loc[kfold['valid_idx0'][0:100000].get(), 'clicks'].compute(), broadcast = True)
        #futures = client.submit(f_, [t1, t2])
        results = client.gather([t1, t2, qid])
        a = xgb.dask.DaskDeviceQuantileDMatrix(client, cudftodask(results[0]), cudftodask(results[1]),
                                    qid = cudftodask(results[2]))
        print(results[2])

In [ ]:
del qid, t1, t2, a, results

In [ ]:
features = val_clicks_cd.columns[2:-1]
testing1 = dask_cudf.from_cudf(val_clicks_cd.loc[kfold['valid_idx0'][0:100000].get(), features].compute(), chunksize = 100)

In [ ]:
%%time

#Train using Dask DF
def trim_memory() -> int:
    libc = ctypes.CDLL("libc.so.6")
    return libc.malloc_trim(0)

def training(cd_dask, cd, features, type_):

    skf = GroupKFold(n_splits=5)
    client.run(gc.collect)
    gc.collect()
    
    #for fold,(train_idx, valid_idx) in enumerate(skf.split(cd_dask, cd_dask[type_], groups=cd_dask['user'])):
    for fold,(train_idx, valid_idx) in enumerate(skf.split(cd_dask, cd_dask[type_], groups=cd_dask['user'])):

        future_qid = client.scatter(cd.loc[train_idx, 'user'])
        train_qid = client.submit(cp.array, future_qid)
        client.run(gc.collect())
        X_train = cd.loc[train_idx, features]
        y_train = cd.loc[train_idx, type_]
        X_valid = cd.loc[valid_idx, features]
        y_valid = cd.loc[valid_idx, type_]
        
        #X_train = dask_cudf.from_dask_dataframe(cd.loc[train_idx, features])
        #y_train = dask_cudf.from_dask_dataframe(cd.loc[train_idx, type_])
        #X_valid = dask_cudf.from_dask_dataframe(cd.loc[valid_idx, features])
        #y_valid = dask_cudf.from_dask_dataframe(cd.loc[valid_idx, type_])

        # IF YOU HAVE 50 CANDIDATE WE USE 50 BELOW
        #dtrain = xgb.dask.DaskDeviceQuantileDMatrix(client, X_train, y_train, qid=train_qid.result())
        #dvalid = xgb.dask.DaskDeviceQuantileDMatrix(client, X_valid, y_valid, qid=train_qid.result())

        #xgb_parms = {'objective':'rank:pairwise', 'tree_method':'gpu_hist'}
        #model = xgb.dask.train(client, xgb_parms, 
        #    dtrain=dtrain,
        #    evals=[(dtrain,'train'),(dvalid,'valid')],
        #    num_boost_round=1000,
        #    verbose_eval=100)
        #model.save_model(f'XGB_fold{fold}_click.xgb')
        #gc.collect()

features = val_clicks_cd.columns[2:-1]
with LocalCUDACluster(CUDA_VISIBLE_DEVICES = '0,1',
                      device_memory_limit=0.9, rmm_pool_size='13GB') as cluster:
    with dask.distributed.Client(cluster) as client:
        client.run(trim_memory)
        training(val_clicks_cd_dask, val_clicks_cd, features, 'clicks')

In [ ]:
#train using DeviceQuantileDMatrix

class IterLoadForDMatrix(xgb.core.DataIter):
    def __init__(self, df=None, features=None, target=None, batch_size=256*1024):
        self.features = features
        self.target = target
        self.df = df
        self.it = 0 # set iterator to 0
        self.batch_size = batch_size
        self.batches = int( np.ceil( len(df) / self.batch_size ) )
        super().__init__()

    def reset(self):
        '''Reset the iterator'''
        self.it = 0

    def next(self, input_data):
        '''Yield next batch of data.'''
        if self.it == self.batches:
            return 0 # Return 0 when there's no more batch.
        
        a = self.it * self.batch_size
        b = min( (self.it + 1) * self.batch_size, len(self.df) )
        #dt = cudf.read_parquet(f'/kaggle/input/ottofeaturesengineering/cd_valid_clicks_p{self.it}.pqt')
        dt = cudf.DataFrame(self.df.iloc[a:b])
        input_data(data=dt[self.features], label=dt[self.target]) #, weight=dt['weight'])
        self.it += 1
        return 1

In [ ]:
'''
%%time
def training(cd, features, type_):

    skf = GroupKFold(n_splits=5)
    
    for fold,(train_idx, valid_idx) in enumerate(skf.split(cd, cd[type_], groups=cd['user'] )):

        train_group = list(cd.loc[train_idx, 'user'].value_counts())
        X_train = cd.loc[train_idx, features]
        y_train = cd.loc[train_idx, type_]
        X_valid = cd.loc[valid_idx, features]
        y_valid = cd.loc[valid_idx, type_]

        # IF YOU HAVE 50 CANDIDATE WE USE 50 BELOW
        dtrain = xgb.DMatrix(X_train, y_train, group=train_group)
        dvalid = xgb.DMatrix(X_valid, y_valid, group=train_group)

        xgb_parms = {'objective':'rank:pairwise', 'tree_method':'gpu_hist'}
        model = xgb.train(xgb_parms, 
            dtrain=dtrain,
            evals=[(dtrain,'train'),(dvalid,'valid')],
            num_boost_round=1000,
            verbose_eval=100)
        model.save_model(f'XGB_fold{fold}_click.xgb')

features = val_clicks_cd.columns[2:-1]
training(val_clicks_cd, features, 'clicks')
'''

In [ ]:
'''
%%time
FEATURES = ['user', 'item', 'item_item_count',
    'item_user_count', 'item_mean_ratio', 'item_click_count', 'item_click2cart_ratio',
    'user_user_count', 'user_item_count', 'user_mean_ratio', 'user_click_count',
    'user_click2cart_ratio', 'clicks']

def training(cd, features, type_):

    skf = GroupKFold(n_splits=5)
    
    for fold,(train_idx, valid_idx) in enumerate(skf.split(cd, cd[type_], groups=cd['user'] )):
        
        
        # TRAIN, VALID, TEST FOR FOLD K
        Xy_train = IterLoadForDMatrix(cd.loc[train_idx], FEATURES, 'type_')
        X_valid = cd.loc[valid_idx, FEATURES]
        y_valid = cd.loc[valid_idx, 'type_']
        
        train_group = list(cd.loc[train_idx, 'user'].value_counts())
        #X_train = cd.loc[train_idx, features]
        #y_train = cd.loc[train_idx, type_]
        #X_valid = cd.loc[valid_idx, features]
        #y_valid = cd.loc[valid_idx, type_]

        # IF YOU HAVE 50 CANDIDATE WE USE 50 BELOW
        dtrain = xgb.DeviceQuantileDMatrix(Xy_train, max_bin=256)
        dvalid = xgb.DMatrix(data=X_valid, label=y_valid)

        xgb_parms = {'objective':'rank:pairwise', 'tree_method':'auto'}
        model = xgb.train(xgb_parms, 
            dtrain=dtrain,
            evals=[(dtrain,'train'),(dvalid,'valid')],
            num_boost_round=1000,
            verbose_eval=100)
        model.save_model(f'XGB_fold{fold}_click.xgb')

features = val_clicks_cd.columns[2:-1]
training(val_clicks_cd, features, 'clicks')
'''